In [2]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd
import psycopg2
import nltk
import re
import string
import sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [3]:
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random)
#filepath = '../Data_sample/FakeNewsCorpus_250.csv' # 250 rows of FakeNewsCorpus
filepath = '../Data_git_ignore/clean-100k.csv'       # 1 mil rows raw
#s = 1000000                                            # desired sample size
#seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0])
df["content"] = df["content"].astype(str)
# create type_id
df['type_id'] = df.groupby(['type']).ngroup()

df.head()

,meta_description,updated_at,content,authors,scraped_at,id,url,meta_keywords,summary,domain,title,keywords,tags,type,inserted_at,source,type_id
0,THE UNIVERSE ceases to exist when we are not l...,2018-02-02 01:19:41.756664,"life is an illusion , at least on a quantum le...",Sean Martin,2018-01-25 16:17:44.789555,2,https://www.express.co.uk/news/science/738402/...,[''],NaN,express.co.uk,Is life an ILLUSION? Researchers prove 'realit...,NaN,NaN,rumor,2018-02-02 01:19:41.756632,NaN,8
1,NaN,2018-02-02 01:19:41.756664,"unfortunately , he hasn t yet attacked her for...","Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",2018-01-25 16:17:44.789555,6,http://barenakedislam.com/category/donald-trum...,[''],NaN,barenakedislam.com,Donald Trump,NaN,NaN,hate,2018-02-02 01:19:41.756632,NaN,4
2,NaN,2018-02-02 01:19:41.756664,the los angeles police department has been den...,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",2018-01-25 16:17:44.789555,7,http://barenakedislam.com/category/donald-trum...,[''],NaN,barenakedislam.com,Donald Trump,NaN,NaN,hate,2018-02-02 01:19:41.756632,NaN,4
3,NaN,2018-02-02 01:19:41.756664,the white house has decided to quietly withdra...,"Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",2018-01-25 16:17:44.789555,8,http://barenakedislam.com/2017/12/24/more-winn...,[''],NaN,barenakedislam.com,"MORE WINNING! Israeli intelligence source, DEB...",NaN,NaN,hate,2018-02-02 01:19:41.756632,NaN,4
4,NaN,2018-02-02 01:19:41.756664,the time has come to cut off the tongues of th...,"F.N. Lehner, Don Spilman, Clarence J. Feinour,...",2018-01-25 16:17:44.789555,9,http://barenakedislam.com/2017/12/25/oh-trump-...,[''],NaN,barenakedislam.com,"“Oh, Trump, you coward, you just wait, we will...",NaN,NaN,hate,2018-02-02 01:19:41.756632,NaN,4


In [10]:
import gensim 
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models.doc2vec import TaggedDocument, Doc2Vec 
from gensim.utils import tokenize
from sklearn.utils import shuffle
q = word_tokenize('bag is up')
q

['bag', 'is', 'up']

In [11]:
#USE THIS
# This can generate a dataset with random purmutation and a max size for each type(can be smaller if desired max is not possible)

# max size for type
max_size = 5000
# traning_set ratio - splits data into traning=ratio,  test and validate=(1-ratio)/2 ex. train=80%, test=10%, validate=10%
ratio=0.8
# Labels to include - ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] - all labels
use_types = ['fake', 'satire', 'bias', 'conspiracy', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable']
# Random seed
rnd = 1

# initialize dataframes
train    = pd.DataFrame(columns = df.columns)
test     = pd.DataFrame(columns = df.columns)

# add type to test splits
for t in use_types:

    # type size
    type_size = df['type'].loc[df['type'] == t].value_counts().min()

    # set size of type slice
    if type_size < max_size:
        tmp = df.loc[df['type'] == t].sample(n = type_size, random_state=rnd)
    else:
        tmp = df.loc[df['type'] == t].sample(n = max_size, random_state=rnd)

    # split current type
    train_tmp, test_tmp = np.split(tmp, [int(ratio * len(tmp))])

    # add tmp to dataframes
    train    = pd.concat([train, train_tmp])
    test     = pd.concat([test, test_tmp])
    
    # print split shape
    print("=>", t, tmp.shape, train_tmp.shape, test_tmp.shape)

print("\n[Final split]\ntrain, test==>", train.shape, test.shape)

=> fake (5000, 17) (4000, 17) (1000, 17)
=> satire (344, 17) (275, 17) (69, 17)
=> bias (5000, 17) (4000, 17) (1000, 17)
=> conspiracy (5000, 17) (4000, 17) (1000, 17)
=> junksci (2204, 17) (1763, 17) (441, 17)
=> hate (298, 17) (238, 17) (60, 17)
=> clickbait (1979, 17) (1583, 17) (396, 17)
=> unreliable (1045, 17) (836, 17) (209, 17)
=> political (5000, 17) (4000, 17) (1000, 17)
=> reliable (289, 17) (231, 17) (58, 17)

[Final split]
train, test==> (20926, 17) (5233, 17)


In [12]:
#train.set_index('id', inplace = True)
train = shuffle(train, random_state = 1)
test = shuffle(test, random_state = 1)
train.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop = True)

In [13]:
%%time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
series_content = train.content
#contTok = series_content.apply(nltk.word_tokenize)
typeLst = train.type_id.tolist()
#np.unique(typeLst, return_counts = True)

Wall time: 0 ns


In [14]:
test_content = test.content
test_typeLst = test.type_id.tolist()

In [15]:
#Try gensim tokenizer
testTok = test_content.apply(tokenize,lowercase =True)
for i in testTok.index:
    testTok[i] = list(testTok[i])

In [16]:
#Try gensim tokenizer
contTok = series_content.apply(tokenize,lowercase =True)
for i in contTok.index:
    contTok[i] = list(contTok[i])

In [17]:
documents = [TaggedDocument(contTok, [i]) for i, contTok in enumerate(contTok)]
#contTok[291496]

In [18]:
%%time
model1 = Doc2Vec(documents, vector_size = 150, min_count = 3, window = 10, epochs=10, workers = 4)

Wall time: 1min 34s


In [19]:
model1.most_similar('apple')

C:\Users\jola1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'apple' not in vocabulary"

In [20]:
model1.docvecs.count

20926

In [21]:
model1.docvecs.most_similar(12000)

[(12463, 0.8173004388809204),
 (11586, 0.8042389154434204),
 (11719, 0.8028494715690613),
 (13643, 0.800359308719635),
 (12523, 0.7767424583435059),
 (12629, 0.7735834121704102),
 (12830, 0.7728123664855957),
 (12908, 0.7709711194038391),
 (11544, 0.769690752029419),
 (12969, 0.7667030096054077)]

In [22]:
#Put all docvecs and labels into arrays for classifier:
train_arrays = np.zeros((model1.docvecs.count,150),dtype="float32")
train_labels = np.zeros(model1.docvecs.count,dtype="float32")
for i in range(model1.docvecs.count):
    train_arrays[i] = model1.docvecs[i]
    train_labels[i] = train.type_id[i]

In [23]:
from sklearn.utils import shuffle
train_arrays, train_labels = shuffle(train_arrays, train_labels)

In [24]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgress")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [25]:
#How infer_vector works:
token = "this is new sentence".split()
new_vector = model1.infer_vector(token)
sims = model1.docvecs.most_similar([new_vector])

In [26]:
#testTok
test_arrays = np.zeros((len(test),150),dtype = "float32")
for i in range(len(test)):
    test_arrays[i] = model1.infer_vector(testTok[i],steps = 15)   
test_labels = np.array(test_typeLst,dtype="float32")

In [24]:
model1.docvecs.most_similar([test_arrays[100]])

[(2990, 0.7172302007675171),
 (2406, 0.631436824798584),
 (2033, 0.6294246315956116),
 (12019, 0.6242984533309937),
 (11307, 0.6152855157852173),
 (150, 0.5990238189697266),
 (1710, 0.5703397989273071),
 (942, 0.5529394149780273),
 (2229, 0.5519653558731079),
 (19678, 0.5503799915313721)]

In [5]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432")

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
s = 250                                            # desired sample size
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT * FROM fakenews.article", connection)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df.head()

,id,domain_id,type_id,url,content,title,meta_description,scraped_at,updated_at,inserted_at
0,437287,1,1,http://beforeitsnews.com/financial-markets/201...,"['got', 'plans', 'saturday', 'of', 'readers', ...",Got Plans Saturday?,None,2018-01-25,2018-02-02,2018-02-02
1,929199,0,0,https://www.christianpost.com/news/birth-of-a-...,"['the', 'views', 'expressed', 'by', 'the', 'au...",Birth of a Muslim Zionist,None,2018-01-25,2018-02-02,2018-02-02
2,433367,1,1,http://beforeitsnews.com/financial-markets/201...,"['headline', 'bitcoin', 'blockchain', 'searche...",Fed to manipulate interest rates (and gold?) t...,None,2018-01-25,2018-02-02,2018-02-02
3,474095,2,0,https://nutritionfacts.org/topics/inflammatory...,"['preventing', 'crohns', 'disease', 'with', 'd...",inflammatory bowel disease,None,2018-01-25,2018-02-02,2018-02-02
4,929765,0,0,https://www.christianpost.com/news/if-free-tra...,"['expand', 'collapse', 'dr', 'gordon', 'borono...","If Free Trade Is So Great, Why Are American Wo...",None,2018-01-25,2018-02-02,2018-02-02


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

df['content'] = df['content'].apply(' '.join)

x = v.fit_transform(df["content"])

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x.toarray(), df["type_id"], test_size=0.2, random_state=0)

In [28]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(12, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [29]:
model.fit(train_arrays, train_labels, epochs=3)

Train on 20926 samples
Epoch 1/3
20926/20926 [==============================] - 1s 35us/sample - loss: 2.1061 - accuracy: 0.1982
Epoch 2/3
20926/20926 [==============================] - 0s 22us/sample - loss: 1.9444 - accuracy: 0.2337
Epoch 3/3
20926/20926 [==============================] - 0s 21us/sample - loss: 1.9178 - accuracy: 0.2507


In [30]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
test_arrays_shuffled, test_labels_shuffled = shuffle(test_arrays, test_labels)

val_loss, val_acc = model.evaluate(test_arrays_shuffled,test_labels_shuffled)
print(val_loss, val_acc)

5233/5233 [==============================] - 0s 25us/sample - loss: 1.9160 - accuracy: 0.2840
1.916020153267261 0.28396714
